# Getting the info that the client needs for D3.js

Often it is easier to transform data in the backend to be used by D3. In general all data in D3 needs to be a list of dicts. In this example I'm using an existing `world` object to demonstrate the methods that do these transformations. 

In [1]:
#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import os
import sys
import numpy as np
import pandas as pd
import pickle
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()


with (open("../pickles/billmanhworld.pkl", 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

In [2]:
sys.path.append('..')
from lib.builders import world as w

`w.get_area_data(world)` gets the area relevant to where the character is. This includes surrounding areas.

In [3]:
world

In [4]:
print(world.Character.location)
pd.DataFrame(w.get_area_data(world)).T

[6, 3]


,aware,danger,elevation,feature,key,nation,nation number,rainfall,terrain,turn_last_visited,visited,x,y
area,0,0.785,70,Stenchforth,6:3,Helllight,5,21,town,0,1,6,3
NArea,0,0.315,87,none,6:2,Helllight,5,22,mountain,0,0,6,2
SArea,0,0.276,56,none,6:4,Helllight,5,23,mountain,0,0,6,4
EArea,0,0.23,75,none,7:3,Helllight,5,32,mountain,0,0,7,3
WArea,0,0.334,61,none,5:3,Helllight,5,13,mountain,0,0,5,3


For other game principals, it's easy to just make regular `pandas` and `numpy` queries on those datasets. 

In [5]:
world.df_features.loc[world.Character.get_location_key(),'visited'] >= 1

True

Querying the terrain is easy as well. For example, here is a query of all of the world where the _terrain_ is _desert_ and the _danger_ is above .5. These queries are fast and cheap. 

In [6]:
world.df_features.loc[(world.df_features['terrain']=='desert') & 
                     (world.df_features['danger']>=.5)].head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
0:2,2,0.0,0,0:2,22.0,desert,NaN,5,Helllight,0,1,0,0.887
0:4,4,0.0,0,0:4,16.0,desert,NaN,5,Helllight,0,1,0,0.831
1:4,4,1.0,1,1:4,20.0,desert,NaN,5,Helllight,0,1,0,0.759
0:5,5,0.0,0,0:5,10.0,desert,NaN,5,Helllight,0,1,0,0.763
1:5,5,1.0,1,1:5,15.0,desert,NaN,5,Helllight,0,1,0,1.041


You can also _mask_ the terrain where the character has no knowledge. 

In [7]:
masked_world = w.mask_unknown(world)
world.Character.title = "Noble"
masked_world.head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
0:0,0,0.0,0,0:0,32.0,unknown,unknown,0,unknown,0,0,0,0.507
1:0,0,0.0,1,1:0,41.0,unknown,unknown,0,unknown,0,0,0,0.470
2:0,0,1.0,2,2:0,47.0,unknown,unknown,0,unknown,0,0,0,0.590
3:0,0,3.0,3,3:0,53.0,unknown,unknown,0,unknown,0,0,0,0.891
4:0,0,6.0,4,4:0,69.0,unknown,unknown,0,unknown,0,0,0,0.745


## Analysis via list comprehension

Querying people is also easy, but you need to use _list comprehension_ to do it. This is the general structure. 

In [56]:
world.culture.eons-(world.towns[0].founded-1000)

10

In [47]:
world.towns[12].population

[Orfrif the Speaker of Stenchforth, Urxand the commoner]

Query the people in the town by setting conditions in the list. These people are the highest in temperment. 

In [9]:
[[(person.name,person.temperment) for person in t.population if person.temperment>.5] for t in world.towns]

[[('Laladdnan', 0.84), ('Akirushottle', 0.56)],
 [('Alifillunt', 0.65)],
 [('Thohoh', 0.8), ('Alresha', 0.9), ('Chifrow', 0.78)],
 [('Erepartath', 0.89), ('Fesassane', 0.61)],
 [('Alirrat', 0.56), ('Xatallun', 0.64)],
 [('Adadsnan', 0.83), ('Amendunder', 0.53), ('Shaplend', 0.66)],
 [],
 [('Chafadder', 0.75), ('Elenkea', 0.69)],
 [('Stitallorg', 0.7), ('Alondud', 0.89)],
 [('Drappselton', 0.52)],
 [('Quispus', 0.77)],
 [('Trucinnich', 0.72), ('Wrakrackoh', 0.8)],
 [('Orfrif', 0.67), ('Urxand', 0.75)],
 [('Mocanel', 0.65)],
 [('Xistot', 0.68)],
 [],
 [('Trangash', 0.74)],
 []]

You can also get summary stats on towns. For example, the average temperment per town.

In [10]:
[(t.name,np.mean([person.temperment for person in t.population])) for t in world.towns]

[('Coremoral', 0.4116666666666666),
 ('Ragering', 0.365),
 ('Camcraft', 0.7424999999999999),
 ('Malcrest', 0.75),
 ('Snakeward', 0.4133333333333334),
 ('Waterblade', 0.556),
 ('Ravenhenge', 0.23500000000000001),
 ('Stenchtear', 0.72),
 ('Demonsborourgh', 0.5766666666666667),
 ('Soilcoil', 0.4033333333333333),
 ('Doomkeep', 0.41500000000000004),
 ('Rockforth', 0.76),
 ('Stenchforth', 0.71),
 ('Rocklair', 0.45333333333333337),
 ('Leafhost', 0.5700000000000001),
 ('Stenchvale', 0.22499999999999998),
 ('Flamerun', 0.32666666666666666),
 ('Clearpike', 0.23)]

For some purposes you may want to get a collection of people and edit them in bulk. This is also possible using list comprehension. 

In [11]:
w.get_people_where_char_has_visited(world)

[{'town': capitol of Stenchforth: population: 2 location: [6,3] founded 1012,
  'people': [Orfrif the Speaker of Stenchforth, Urxand the commoner]}]

In [12]:
r = w.get_people_where_char_has_visited(world)[0]
r

{'town': capitol of Stenchforth: population: 2 location: [6,3] founded 1012,
 'people': [Orfrif the Speaker of Stenchforth, Urxand the commoner]}

In [13]:
r['town'].nation

'Stenchforth'

In [14]:
w.get_relationships_node_map(world)

[{'name': 'Stenchforth',
  'title': 'capitol of Stenchforth',
  'nation': 'Stenchforth',
  'population': 2,
  'type': 'capitol',
  'location': '6:3',
  'founded year': 1012,
  'children': [{'name': 'Orfrif',
    'role': 'Speaker of Stenchforth',
    'temperment': 0.67,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Urxand',
    'role': 'commoner',
    'temperment': 0.75,
    'loyalty': 0.5,
    'type': 'person'}]}]

## Getting map data (mapData): 

In [15]:
def coordkey(coord):
    '''
    takes a coord [1,1], returns key "1:1"
    '''
    key = ":".join([str(i) for i in coord])
    return key


def get_features_or_NA(world,coord):
    try:
        l = world.df_features.loc[
                chordKey([
                    int(coord[0]),int(coord[1])
                    ])
                ].fillna("none").to_dict(),
    except: 
        l = [{"terrain":"void"}]
    return l[0]


def get_area_data(world):
    """
    note: requires Character
    """
    mapData = {}
    coord = world.Character.location
    key = world.Character.get_location_key()
    l = world.df_features.loc[key]
    mapData = {'area':world.df_features.loc[key].fillna("none").to_dict(),
              'NArea':get_features_or_NA(world,[coord[0],coord[1]-1]),
              'SArea':get_features_or_NA(world,[coord[0],coord[1]+1]),
              'EArea':get_features_or_NA(world,[coord[0]+1,coord[1]]),
              'WArea':get_features_or_NA(world,[coord[0]-1,coord[1]])}
    return mapData


get_area_data(world)

{'area': {'y': 3,
  'rainfall': 21.0,
  'x': 6,
  'key': '6:3',
  'elevation': 70.0,
  'terrain': 'town',
  'feature': 'Stenchforth',
  'nation number': 5,
  'nation': 'Helllight',
  'visited': 1,
  'aware': 0,
  'turn_last_visited': 0,
  'danger': 0.785},
 'NArea': {'terrain': 'void'},
 'SArea': {'terrain': 'void'},
 'EArea': {'terrain': 'void'},
 'WArea': {'terrain': 'void'}}

In [16]:
world.df_features.loc[
                chordKey([
                    int(coord[0]),int(coord[1])
                    ])
                ]

NameError: name 'chordKey' is not defined

In [ ]:
get_features_or_NA(world,[coord[0],coord[1]-1])

In [ ]:
nations_where_character_has_been = np.unique(where_the_char_has_been['nation'].dropna())
nations_where_character_has_been

You can use this to get lists of people who the character knows. 

In [ ]:
[[t for t in n.get_all_towns(world)] for n in world.nations if n.name in nations_where_character_has_been]

In [ ]:
[[t for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

In [ ]:
[(T.nation,T.name) for T in world.towns]

In [ ]:
[[t.get_person_data() for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

In [ ]:
masked_world.loc[world.Character.get_location_key()]

You can query nations or towns. `world.nations` is a list of _nations_ and `world.towns` is a list of _towns_

In [ ]:
where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                (world.df_features['terrain']=='town')]
where_the_char_has_been

## Getting the diplomacy and relationships data for the journal page

In [ ]:
rel = w.get_people_where_char_has_visited(world)
rel

In [ ]:
w.get_relationships_node_map(world)

In [ ]:
hierarchy = [{'nation':i.name,
              'children':i.diplomacy} for i in world.nations]



In [ ]:
p = rel[0]['town'].population[0]


In [ ]:
def get_people_where_char_has_visited(world):
    """
    returns a dictionary of {{"town":[<obj>],"people":[<obj>]}
    filtered to where the character has been. 
    """
    where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                    (world.df_features['terrain']=='town')].dropna()
    towns_and_people = [{"town":T,
                         "people":[t for t in T.population]} 
                        for T in world.towns if T.name in np.unique(w.where_the_char_has_been['feature'])]
    return towns_and_people

def get_relationships_node_map(world):
    nodes = [{'name':r['town'].name,
              'title':str(r['town']).split(":")[0],
              'nation':r['town'].diplomacy['nation'],
              'population':r['town'].pop,
              'type':r['town'].type,
              'location':r['town'].key,
              'founded year':r['town'].founded,
              'national fealty':r['town'].diplomacy['national fealty'],
             'children': [{'name':p.name,
                          'role':p.role,
                          'temperment':p.temperment,
                          'loyalty':p.loyalty,
                          'type':'person'} for p in r['people']]} 
             for r in w.get_people_where_char_has_visited(world)]
    return nodes
        
w.get_people_where_char_has_visited(world)

In [ ]:
relationships = w.get_relationships_node_map(world)
relationships

In [ ]:
nations = [s['nation'] for s in relationships]
nations

In [ ]:
[i for i in relationships]

In [ ]:
[{'nation':i,'towns':[t for t in relationships if t['nation']==i]} for i in nations]

In [ ]:
[i for i in nations]